In [2]:
import warnings

from pathlib import Path
from functools import lru_cache

from dotenv import load_dotenv
from IPython.display import Markdown, display

from persuasion_bias.rag.chains import CompressionRAGChain
from persuasion_bias.config.logging import LogCallback

warnings.filterwarnings("ignore")
_ = load_dotenv(override=True)

config_file = Path("persuasion_bias") / "config" / "log-config.yaml"
logger_cb = LogCallback()
logger_cb.reconfigure_from_file(config_file=config_file)
log = logger_cb.logger

In [3]:
compressionRAG = CompressionRAGChain(log=log)

[ERROR @LogCallback]: You must specify either `repo_id` or set `use_anthropic`.


[STDERR: ERROR 2025-09-12 18:46:50,122 chains.py:168]


ValueError: You must specify either `repo_id` or set `use_anthropic`.

In [5]:
compressionRAG = CompressionRAGChain(log=log, use_anthropic=True)

log.info(compressionRAG.__doc__)

[INFO @LogCallback]: Using Anthropic model: claude-3-5-haiku-20241022
[INFO @LogCallback]: 
    +---------------------------------------------------------------+
    | One-turn RAG runnable with contextual compression retrieval.  |
    | Optionally selects:                                           |
    |       - Claude 3.5 Haiku                                      |
    |       - similarity_threshold for relevance filtering          |
    +---------------------------------------------------------------+
    


In [6]:
compression_retriever = compressionRAG._get_contextual_compression_retriever(similarity_threshold=0.45)
compressionRAG._retriever = compression_retriever

runnable_seq = compressionRAG.create_runnable_sequence()

[INFO @LogCallback]: Creating contextual compression retriever
[INFO @sentence_transformers.SentenceTransformer]: Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
[INFO @chromadb.telemetry.product.posthog]: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
[INFO @LogCallback]: Retriever created and cached


In [7]:
@lru_cache(maxsize=16)
def argue(argument: str) -> None:
    """Invoke runnable sequence -  Markdown display"""

    log.info("Invoking the model\n")
    try:
        response = runnable_seq.invoke(argument)
        display(Markdown(response))
    except Exception as e:
        log.exception(e)

In [8]:
argue(argument="Social media is harmful for the youth.")

[INFO @LogCallback]: Invoking the model

[INFO @httpx]: HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


## Argument Analysis: Social Media's Impact on Youth

### Bias Detection
- **Bias Score**: 7/10 (High potential for bias)
- **Key Bias Indicators**:
  - Emotional language
  - Generalized claims
  - Lack of nuanced perspective

### Argument Strengths
1. Developmental concerns
2. Mental health risks
3. Addiction potential

### Argument Weaknesses
1. Overgeneralization
2. Ignores potential benefits
3. Lacks individual variability

### Recommended Balanced Perspective
- Moderate, supervised technology use
- Digital literacy education
- Individual monitoring
- Balanced screen time

### Conclusion
While social media presents risks, a blanket condemnation oversimplifies a complex issue. Targeted, informed approaches are more effective than total restriction.

### Evidence Base
- Multiple AI-generated documents suggest:
  - Correlation between social media and mental health issues
  - Developmental vulnerability of youth
  - Need for guided technology interaction

**Recommendation**: Implement structured, age-appropriate digital engagement strategies.